In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd drive/MyDrive/HiWi/Aircraft_Noise

In [ ]:
! pip install  typing traffic

In [ ]:
! pip install pandas joblib openpyxl matplotlib



In [ ]:
!pip install folium


In [ ]:
!pip install ipywidgets


# Loading ADS-B Data

In [ ]:
import joblib
import pandas as pd

# Load ADS-B Data (Trajectory Data)
adsb_data = joblib.load("data_2022_january.joblib")

df_adsb = adsb_data.data

In [ ]:
print("ADS-B Data Loaded:")
display(df_adsb.info())

In [ ]:
display(df_adsb.head())

# Export processed ADS-B data to CSV

In [ ]:
#If needed , proccessed adsb data can be downloaded as a CSV file
df_adsb.to_csv("processed_adsb_data.csv", index=False)
print("ADS-B data exported to 'processed_adsb_data.csv'")

from google.colab import files
files.download("processed_adsb_data.csv")


In [ ]:
# Extract the date part
df_adsb["date"] = df_adsb["timestamp"].dt.date

# Sort by time (to maintain trajectory order)
df_adsb = df_adsb.sort_values(by=["icao24", "date", "timestamp"])

display(df_adsb.head())

# Grouped flight summary

In [ ]:
# Unique flight identifier by grouping by (icao24, date)
df_adsb["flight_id"] = df_adsb.groupby(["icao24", "date"]).ngroup()

In [ ]:
grouped_data = df_adsb.groupby("flight_id").agg({
    "icao24": "first",
    "callsign": "first",
    "date": "first",
    "timestamp": ["min", "max"]
}).reset_index()

grouped_data.columns = ["flight_id", "icao24", "callsign", "date", "first_seen", "last_seen"]

print("Grouped flight data:")
display(grouped_data.head())


# Getting Trajectories

In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import folium

# --- Dropdown Widgets ---
icao24_list = sorted(df_adsb["icao24"].dropna().unique())

icao24_dropdown = widgets.Dropdown(
    options=icao24_list,
    description="Select ICAO24:",
    layout=widgets.Layout(width='50%')
)

timestamp_dropdown = widgets.Dropdown(
    options=[],
    description="Select Date:",
    layout=widgets.Layout(width='50%')
)

plot_type_dropdown = widgets.Dropdown(
    options=["Simple Graph", "Folium Map", "Both"],
    description="Plot Type:",
    layout=widgets.Layout(width='50%')
)

output = widgets.Output()

# --- Helper Functions ---
def get_flight_id(icao24, date):
    result = df_adsb[(df_adsb["icao24"] == icao24) & (df_adsb["date"] == date)]
    if result.empty:
        return None
    return result["flight_id"].unique()[0]  # One flight per icao24+date assumed

def plot_matplotlib_trajectory(flight_data, flight_id):
    icao24 = flight_data["icao24"].iloc[0]
    callsign = flight_data["callsign"].iloc[0]
    firstseen = flight_data["timestamp"].min()
    lastseen = flight_data["timestamp"].max()

    plt.figure(figsize=(10, 6))
    plt.plot(flight_data["longitude"], flight_data["latitude"], marker="o", linestyle="-", markersize=4,color="#FF10F0")
    title = (
        f"Flight ID: {flight_id} | ICAO24: {icao24} | Callsign: {callsign}\n"
        f"First Seen: {firstseen} | Last Seen: {lastseen}"
    )
    plt.title(title, fontsize=11)
    plt.annotate("Start", (flight_data["longitude"].iloc[0], flight_data["latitude"].iloc[0]), fontsize=9, color='green')
    plt.annotate("End", (flight_data["longitude"].iloc[-1], flight_data["latitude"].iloc[-1]), fontsize=9, color='red')
    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def plot_folium_trajectory(flight_data, flight_id):
    icao24 = flight_data["icao24"].iloc[0]
    callsign = flight_data["callsign"].iloc[0]
    firstseen = flight_data["timestamp"].min()
    lastseen = flight_data["timestamp"].max()

    start_coords = [flight_data["latitude"].iloc[0], flight_data["longitude"].iloc[0]]
    m = folium.Map(location=start_coords, zoom_start=7)
    coords = list(zip(flight_data["latitude"], flight_data["longitude"]))
    folium.PolyLine(coords, color='#FF10F0', weight=3, opacity=0.8).add_to(m)
    folium.Marker(coords[0], popup="Start", icon=folium.Icon(color='green')).add_to(m)
    folium.Marker(coords[-1], popup="End", icon=folium.Icon(color='red')).add_to(m)

    folium.map.Marker(
        start_coords,
        icon=folium.DivIcon(
            html=f"""
            <div style="font-size: 12pt">
                <b>Flight ID:</b> {flight_id}<br>
                <b>ICAO24:</b> {icao24}<br>
                <b>Callsign:</b> {callsign}<br>
                <b>First Seen:</b> {firstseen}<br>
                <b>Last Seen:</b> {lastseen}
            </div>
            """
        )
    ).add_to(m)

    return m

# --- Widget Event Handlers ---
def on_icao24_change(change):
    icao24 = change['new']
    matching_dates = df_adsb[df_adsb["icao24"] == icao24]["date"].dropna().unique()
    timestamp_dropdown.options = sorted(matching_dates)

def on_plot_button_clicked(b):
    with output:
        clear_output()
        icao24 = icao24_dropdown.value
        date = timestamp_dropdown.value
        plot_type = plot_type_dropdown.value
        flight_id = get_flight_id(icao24, date)

        if flight_id is None:
            print("No flight data found for the selection.")
            return

        flight_data = df_adsb[df_adsb["flight_id"] == flight_id]

        if plot_type == "Simple Graph":
            plot_matplotlib_trajectory(flight_data, flight_id)
        elif plot_type == "Folium Map":
            m = plot_folium_trajectory(flight_data, flight_id)
            display(m)
        elif plot_type == "Both":
            plot_matplotlib_trajectory(flight_data, flight_id)
            m = plot_folium_trajectory(flight_data, flight_id)
            display(m)

# --- Plot Button ---
plot_button = widgets.Button(description="Plot Flight", button_style='primary')

# Attach handlers
icao24_dropdown.observe(on_icao24_change, names='value')
plot_button.on_click(on_plot_button_clicked)

# Initial trigger
on_icao24_change({'new': icao24_dropdown.value})

# --- Display UI ---
display(icao24_dropdown, timestamp_dropdown, plot_type_dropdown, plot_button, output)


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import folium
from mpl_toolkits.mplot3d import Axes3D  # For 3D plotting

# --- Dropdown Widgets ---
icao24_list = sorted(df_adsb["icao24"].dropna().unique())

icao24_dropdown = widgets.Dropdown(
    options=icao24_list,
    description="Select ICAO24:",
    layout=widgets.Layout(width='50%')
)

timestamp_dropdown = widgets.Dropdown(
    options=[],
    description="Select Date:",
    layout=widgets.Layout(width='50%')
)

plot_type_dropdown = widgets.Dropdown(
    options=["Simple Graph", "Folium Map", "3D Plot", "Both", "All (2D + 3D + Map)"],
    description="Plot Type:",
    layout=widgets.Layout(width='50%')
)

output = widgets.Output()

# --- Helper Functions ---
def get_flight_id(icao24, date):
    result = df_adsb[(df_adsb["icao24"] == icao24) & (df_adsb["date"] == date)]
    if result.empty:
        return None
    return result["flight_id"].unique()[0]

def plot_matplotlib_trajectory(flight_data, flight_id):
    icao24 = flight_data["icao24"].iloc[0]
    callsign = flight_data["callsign"].iloc[0]
    firstseen = flight_data["timestamp"].min()
    lastseen = flight_data["timestamp"].max()

    plt.figure(figsize=(10, 6))
    plt.plot(flight_data["longitude"], flight_data["latitude"], marker="o", linestyle="-", markersize=4, color="#FF10F0")
    title = (
        f"Flight ID: {flight_id} | ICAO24: {icao24} | Callsign: {callsign}\n"
        f"First Seen: {firstseen} | Last Seen: {lastseen}"
    )
    plt.title(title, fontsize=11)
    plt.annotate("Start", (flight_data["longitude"].iloc[0], flight_data["latitude"].iloc[0]), fontsize=9, color='green')
    plt.annotate("End", (flight_data["longitude"].iloc[-1], flight_data["latitude"].iloc[-1]), fontsize=9, color='red')

    box_text = (
        f"Flight ID: {flight_id}\n"
        f"ICAO24: {icao24}\n"
        f"Callsign: {callsign}\n"
        f"First Seen: {firstseen}\n"
        f"Last Seen: {lastseen}"
    )
    plt.gca().text(
        0.02, 0.98, box_text,
        transform=plt.gca().transAxes,
        fontsize=10,
        verticalalignment='top',
        bbox=dict(boxstyle="round,pad=0.5", facecolor="white", edgecolor="#FF10F0", alpha=0.8)
    )

    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def plot_3d_trajectory(flight_data, flight_id):
    icao24 = flight_data["icao24"].iloc[0]
    callsign = flight_data["callsign"].iloc[0]
    firstseen = flight_data["timestamp"].min()
    lastseen = flight_data["timestamp"].max()

    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot(
        flight_data["longitude"],
        flight_data["latitude"],
        flight_data["altitude"],
        color="#FF10F0",
        linewidth=2,
        marker='o',
        markersize=3
    )

    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_zlabel("Altitude (m)")
    ax.set_title("3D Flight Trajectory", fontsize=14, pad=20)

    box_text = (
        f"Flight ID: {flight_id}\n"
        f"ICAO24: {icao24} | Callsign: {callsign}\n"
        f"First Seen: {firstseen}\n"
        f"Last Seen: {lastseen}"
    )
    ax.text2D(0.02, 0.95, box_text, transform=ax.transAxes,
              bbox=dict(boxstyle="round,pad=0.5", facecolor="white", edgecolor="#FF10F0", alpha=0.9),
              fontsize=10)
    plt.tight_layout()
    plt.show()

def plot_folium_trajectory(flight_data, flight_id):
    icao24 = flight_data["icao24"].iloc[0]
    callsign = flight_data["callsign"].iloc[0]
    firstseen = flight_data["timestamp"].min()
    lastseen = flight_data["timestamp"].max()

    start_coords = [flight_data["latitude"].iloc[0], flight_data["longitude"].iloc[0]]
    m = folium.Map(location=start_coords, zoom_start=7)
    coords = list(zip(flight_data["latitude"], flight_data["longitude"]))
    folium.PolyLine(coords, color='#FF10F0', weight=3, opacity=0.8).add_to(m)
    folium.Marker(coords[0], popup="Start", icon=folium.Icon(color='green')).add_to(m)
    folium.Marker(coords[-1], popup="End", icon=folium.Icon(color='red')).add_to(m)

    info_html = f"""
    <div style="background-color:white; border:2px solid #FF10F0; border-radius:10px; padding:10px;">
        <b>Flight ID:</b> {flight_id}<br>
        <b>ICAO24:</b> {icao24}<br>
        <b>Callsign:</b> {callsign}<br>
        <b>First Seen:</b> {firstseen}<br>
        <b>Last Seen:</b> {lastseen}
    </div>
    """
    folium.Marker(
        location=start_coords,
        icon=folium.DivIcon(html=info_html)
    ).add_to(m)

    return m

# --- Widget Event Handlers ---
def on_icao24_change(change):
    icao24 = change['new']
    matching_dates = df_adsb[df_adsb["icao24"] == icao24]["date"].dropna().unique()
    timestamp_dropdown.options = sorted(matching_dates)

def on_plot_button_clicked(b):
    with output:
        clear_output()
        icao24 = icao24_dropdown.value
        date = timestamp_dropdown.value
        plot_type = plot_type_dropdown.value
        flight_id = get_flight_id(icao24, date)

        if flight_id is None:
            print("No flight data found for the selection.")
            return

        flight_data = df_adsb[df_adsb["flight_id"] == flight_id]

        if plot_type == "Simple Graph":
            plot_matplotlib_trajectory(flight_data, flight_id)
        elif plot_type == "Folium Map":
            m = plot_folium_trajectory(flight_data, flight_id)
            display(m)
        elif plot_type == "3D Plot":
            plot_3d_trajectory(flight_data, flight_id)
        elif plot_type == "Both":
            plot_matplotlib_trajectory(flight_data, flight_id)
            m = plot_folium_trajectory(flight_data, flight_id)
            display(m)
        elif plot_type == "All (2D + 3D + Map)":
            plot_matplotlib_trajectory(flight_data, flight_id)
            plot_3d_trajectory(flight_data, flight_id)
            m = plot_folium_trajectory(flight_data, flight_id)
            display(m)

# --- Plot Button ---
plot_button = widgets.Button(description="Plot Flight", button_style='primary')

# Attach handlers
icao24_dropdown.observe(on_icao24_change, names='value')
plot_button.on_click(on_plot_button_clicked)

# Initial trigger
on_icao24_change({'new': icao24_dropdown.value})

# --- Display UI ---
display(icao24_dropdown, timestamp_dropdown, plot_type_dropdown, plot_button, output)


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import folium
from mpl_toolkits.mplot3d import Axes3D  # For 3D plotting

# --- Dropdown Widgets ---
icao24_list = sorted(df_adsb["icao24"].dropna().unique())

icao24_dropdown = widgets.Dropdown(
    options=icao24_list,
    description="Select ICAO24:",
    layout=widgets.Layout(width='50%')
)

timestamp_dropdown = widgets.Dropdown(
    options=[],
    description="Select Date:",
    layout=widgets.Layout(width='50%')
)

plot_type_dropdown = widgets.Dropdown(
    options=["Simple Graph", "Folium Map", "3D Plot", "Both", "All (2D + 3D + Map)"],
    description="Plot Type:",
    layout=widgets.Layout(width='50%')
)

output = widgets.Output()

# --- Helper Functions ---
def get_flight_id(icao24, date):
    result = df_adsb[(df_adsb["icao24"] == icao24) & (df_adsb["date"] == date)]
    if result.empty:
        return None
    return result["flight_id"].unique()[0]

def plot_matplotlib_trajectory(flight_data, flight_id):
    icao24 = flight_data["icao24"].iloc[0]
    callsign = flight_data["callsign"].iloc[0]
    firstseen = flight_data["timestamp"].min()
    lastseen = flight_data["timestamp"].max()

    plt.figure(figsize=(10, 6))
    plt.plot(flight_data["longitude"], flight_data["latitude"], marker="o", linestyle="-", markersize=4, color="#FF10F0")
    title = (
        f"Flight ID: {flight_id} | ICAO24: {icao24} | Callsign: {callsign}\n"
        f"First Seen: {firstseen} | Last Seen: {lastseen}"
    )
    plt.title(title, fontsize=11)
    plt.annotate("Start", (flight_data["longitude"].iloc[0], flight_data["latitude"].iloc[0]), fontsize=9, color='green')
    plt.annotate("End", (flight_data["longitude"].iloc[-1], flight_data["latitude"].iloc[-1]), fontsize=9, color='red')

    box_text = (
        f"Flight ID: {flight_id}\n"
        f"ICAO24: {icao24}\n"
        f"Callsign: {callsign}\n"
        f"First Seen: {firstseen}\n"
        f"Last Seen: {lastseen}"
    )
    plt.gca().text(
        0.02, 0.98, box_text,
        transform=plt.gca().transAxes,
        fontsize=10,
        verticalalignment='top',
        bbox=dict(boxstyle="round,pad=0.5", facecolor="white", edgecolor="#FF10F0", alpha=0.8)
    )

    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def plot_3d_trajectory(flight_data, flight_id):
    icao24 = flight_data["icao24"].iloc[0]
    callsign = flight_data["callsign"].iloc[0]
    firstseen = flight_data["timestamp"].min()
    lastseen = flight_data["timestamp"].max()

    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot(
        flight_data["longitude"],
        flight_data["latitude"],
        flight_data["altitude"],
        color="#FF10F0",
        linewidth=2,
        marker='o',
        markersize=3
    )

    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_zlabel("Altitude (m)")
    ax.set_title("3D Flight Trajectory", fontsize=14, pad=20)

    box_text = (
        f"Flight ID: {flight_id}\n"
        f"ICAO24: {icao24} | Callsign: {callsign}\n"
        f"First Seen: {firstseen}\n"
        f"Last Seen: {lastseen}"
    )
    ax.text2D(0.02, 0.95, box_text, transform=ax.transAxes,
              bbox=dict(boxstyle="round,pad=0.5", facecolor="white", edgecolor="#FF10F0", alpha=0.9),
              fontsize=10)
    plt.tight_layout()
    plt.show()

def plot_folium_trajectory(flight_data, flight_id):
    icao24 = flight_data["icao24"].iloc[0]
    callsign = flight_data["callsign"].iloc[0]
    firstseen = flight_data["timestamp"].min()
    lastseen = flight_data["timestamp"].max()

    start_coords = [flight_data["latitude"].iloc[0], flight_data["longitude"].iloc[0]]
    m = folium.Map(location=start_coords, zoom_start=7)
    coords = list(zip(flight_data["latitude"], flight_data["longitude"]))
    folium.PolyLine(coords, color='#FF10F0', weight=3, opacity=0.8).add_to(m)
    folium.Marker(coords[0], popup="Start", icon=folium.Icon(color='green')).add_to(m)
    folium.Marker(coords[-1], popup="End", icon=folium.Icon(color='red')).add_to(m)

    info_html = f"""
    <div style="background-color:white; border:2px solid #FF10F0; border-radius:10px; padding:10px;">
        <b>Flight ID:</b> {flight_id}<br>
        <b>ICAO24:</b> {icao24}<br>
        <b>Callsign:</b> {callsign}<br>
        <b>First Seen:</b> {firstseen}<br>
        <b>Last Seen:</b> {lastseen}
    </div>
    """
    folium.Marker(
        location=start_coords,
        icon=folium.DivIcon(html=info_html)
    ).add_to(m)

    return m

# --- Widget Event Handlers ---
def on_icao24_change(change):
    icao24 = change['new']
    matching_dates = df_adsb[df_adsb["icao24"] == icao24]["date"].dropna().unique()
    timestamp_dropdown.options = sorted(matching_dates)

def on_plot_button_clicked(b):
    with output:
        clear_output()
        icao24 = icao24_dropdown.value
        date = timestamp_dropdown.value
        plot_type = plot_type_dropdown.value
        flight_id = get_flight_id(icao24, date)

        if flight_id is None:
            print("No flight data found for the selection.")
            return

        flight_data = df_adsb[df_adsb["flight_id"] == flight_id]

        if plot_type == "Simple Graph":
            plot_matplotlib_trajectory(flight_data, flight_id)
        elif plot_type == "Folium Map":
            m = plot_folium_trajectory(flight_data, flight_id)
            display(m)
        elif plot_type == "3D Plot":
            plot_3d_trajectory(flight_data, flight_id)
        elif plot_type == "Both":
            plot_matplotlib_trajectory(flight_data, flight_id)
            m = plot_folium_trajectory(flight_data, flight_id)
            display(m)
        elif plot_type == "All (2D + 3D + Map)":
            plot_matplotlib_trajectory(flight_data, flight_id)
            plot_3d_trajectory(flight_data, flight_id)
            m = plot_folium_trajectory(flight_data, flight_id)
            display(m)

# --- Plot Button ---
plot_button = widgets.Button(description="Plot Flight", button_style='primary')

# Attach handlers
icao24_dropdown.observe(on_icao24_change, names='value')
plot_button.on_click(on_plot_button_clicked)

# Initial trigger
on_icao24_change({'new': icao24_dropdown.value})

# --- Display UI ---
display(icao24_dropdown, timestamp_dropdown, plot_type_dropdown, plot_button, output)


In [ ]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import folium
from mpl_toolkits.mplot3d import Axes3D  # For 3D plotting

# --- Dropdown Widgets ---
icao24_list = sorted(df_adsb["icao24"].dropna().unique())

icao24_dropdown = widgets.Dropdown(
    options=icao24_list,
    description="Select ICAO24:",
    layout=widgets.Layout(width='50%')
)

timestamp_dropdown = widgets.Dropdown(
    options=[],
    description="Select Date:",
    layout=widgets.Layout(width='50%')
)

plot_type_dropdown = widgets.Dropdown(
    options=["Simple Graph", "Folium Map", "3D Plot", "Both", "All (2D + 3D + Map)"],
    description="Plot Type:",
    layout=widgets.Layout(width='50%')
)

output = widgets.Output()

# --- Helper Functions ---
def get_flight_id(icao24, date):
    result = df_adsb[(df_adsb["icao24"] == icao24) & (df_adsb["date"] == date)]
    if result.empty:
        return None
    return result["flight_id"].unique()[0]

def plot_matplotlib_trajectory(flight_data, flight_id):
    icao24 = flight_data["icao24"].iloc[0]
    callsign = flight_data["callsign"].iloc[0]
    firstseen = flight_data["timestamp"].min()
    lastseen = flight_data["timestamp"].max()

    plt.figure(figsize=(10, 6))
    plt.plot(flight_data["longitude"], flight_data["latitude"], marker="o", linestyle="-", markersize=4, color="#FF10F0")
    title = (
        f"Flight ID: {flight_id} | ICAO24: {icao24} | Callsign: {callsign}\n"
        f"First Seen: {firstseen} | Last Seen: {lastseen}"
    )
    plt.title(title, fontsize=11)
    plt.annotate("Start", (flight_data["longitude"].iloc[0], flight_data["latitude"].iloc[0]), fontsize=9, color='green')
    plt.annotate("End", (flight_data["longitude"].iloc[-1], flight_data["latitude"].iloc[-1]), fontsize=9, color='red')

    box_text = (
        f"Flight ID: {flight_id}\n"
        f"ICAO24: {icao24}\n"
        f"Callsign: {callsign}\n"
        f"First Seen: {firstseen}\n"
        f"Last Seen: {lastseen}"
    )
    plt.gca().text(
        0.02, 0.98, box_text,
        transform=plt.gca().transAxes,
        fontsize=10,
        verticalalignment='top',
        bbox=dict(boxstyle="round,pad=0.5", facecolor="white", edgecolor="#FF10F0", alpha=0.8)
    )

    plt.xlabel("Longitude")
    plt.ylabel("Latitude")
    plt.grid(True)
    plt.tight_layout()
    plt.show()

def plot_3d_trajectory(flight_data, flight_id):
    icao24 = flight_data["icao24"].iloc[0]
    callsign = flight_data["callsign"].iloc[0]
    firstseen = flight_data["timestamp"].min()
    lastseen = flight_data["timestamp"].max()

    fig = plt.figure(figsize=(12, 8))
    ax = fig.add_subplot(111, projection='3d')
    ax.plot(
        flight_data["longitude"],
        flight_data["latitude"],
        flight_data["altitude"],
        color="#FF10F0",
        linewidth=2,
        marker='o',
        markersize=3
    )

    ax.set_xlabel("Longitude")
    ax.set_ylabel("Latitude")
    ax.set_zlabel("Altitude (m)")
    ax.set_title("3D Flight Trajectory", fontsize=14, pad=20)

    box_text = (
        f"Flight ID: {flight_id}\n"
        f"ICAO24: {icao24} | Callsign: {callsign}\n"
        f"First Seen: {firstseen}\n"
        f"Last Seen: {lastseen}"
    )
    ax.text2D(0.02, 0.95, box_text, transform=ax.transAxes,
              bbox=dict(boxstyle="round,pad=0.5", facecolor="white", edgecolor="#FF10F0", alpha=0.9),
              fontsize=10)
    plt.tight_layout()
    plt.show()

def plot_folium_trajectory(flight_data, flight_id):
    icao24 = flight_data["icao24"].iloc[0]
    callsign = flight_data["callsign"].iloc[0]
    firstseen = flight_data["timestamp"].min()
    lastseen = flight_data["timestamp"].max()

    start_coords = [flight_data["latitude"].iloc[0], flight_data["longitude"].iloc[0]]
    m = folium.Map(location=start_coords, zoom_start=7)
    coords = list(zip(flight_data["latitude"], flight_data["longitude"]))
    folium.PolyLine(coords, color='#FF10F0', weight=3, opacity=0.8).add_to(m)
    folium.Marker(coords[0], popup="Start", icon=folium.Icon(color='green')).add_to(m)
    folium.Marker(coords[-1], popup="End", icon=folium.Icon(color='red')).add_to(m)

    info_html = f"""
    <div style="background-color:white; border:2px solid #FF10F0; border-radius:10px; padding:10px;">
        <b>Flight ID:</b> {flight_id}<br>
        <b>ICAO24:</b> {icao24}<br>
        <b>Callsign:</b> {callsign}<br>
        <b>First Seen:</b> {firstseen}<br>
        <b>Last Seen:</b> {lastseen}
    </div>
    """
    folium.Marker(
        location=start_coords,
        icon=folium.DivIcon(html=info_html)
    ).add_to(m)

    return m

# --- Widget Event Handlers ---
def on_icao24_change(change):
    icao24 = change['new']
    matching_dates = df_adsb[df_adsb["icao24"] == icao24]["date"].dropna().unique()
    timestamp_dropdown.options = sorted(matching_dates)

def on_plot_button_clicked(b):
    with output:
        clear_output()
        icao24 = icao24_dropdown.value
        date = timestamp_dropdown.value
        plot_type = plot_type_dropdown.value
        flight_id = get_flight_id(icao24, date)

        if flight_id is None:
            print("No flight data found for the selection.")
            return

        flight_data = df_adsb[df_adsb["flight_id"] == flight_id]

        if plot_type == "Simple Graph":
            plot_matplotlib_trajectory(flight_data, flight_id)
        elif plot_type == "Folium Map":
            m = plot_folium_trajectory(flight_data, flight_id)
            display(m)
        elif plot_type == "3D Plot":
            plot_3d_trajectory(flight_data, flight_id)
        elif plot_type == "Both":
            plot_matplotlib_trajectory(flight_data, flight_id)
            m = plot_folium_trajectory(flight_data, flight_id)
            display(m)
        elif plot_type == "All (2D + 3D + Map)":
            plot_matplotlib_trajectory(flight_data, flight_id)
            plot_3d_trajectory(flight_data, flight_id)
            m = plot_folium_trajectory(flight_data, flight_id)
            display(m)

# --- Plot Button ---
plot_button = widgets.Button(description="Plot Flight", button_style='primary')

# Attach handlers
icao24_dropdown.observe(on_icao24_change, names='value')
plot_button.on_click(on_plot_button_clicked)

# Initial trigger
on_icao24_change({'new': icao24_dropdown.value})

# --- Display UI ---
display(icao24_dropdown, timestamp_dropdown, plot_type_dropdown, plot_button, output)
